In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

degree_jobs_cleaned = pd.read_csv("degree_jobs_cleaned.csv")
degree_jobs_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 759 entries, 0 to 758
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   job_id             759 non-null    int64 
 1   job_title          759 non-null    object
 2   salary_usd         759 non-null    int64 
 3   employment_type    759 non-null    object
 4   company_location   759 non-null    object
 5   is_remote          759 non-null    bool  
 6   employee_location  759 non-null    object
 7   job_skills         759 non-null    object
 8   degree_required    759 non-null    object
 9   posting_date       759 non-null    object
 10  company_name       759 non-null    object
 11  has_target         759 non-null    bool  
 12  salary_tier        759 non-null    object
 13  degree_flag        759 non-null    object
dtypes: bool(2), int64(2), object(10)
memory usage: 72.8+ KB


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

no_degree_jobs_cleaned = pd.read_csv("no_degree_jobs_cleaned.csv")
no_degree_jobs_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1184 entries, 0 to 1183
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   job_id             1184 non-null   int64 
 1   job_title          1184 non-null   object
 2   salary_usd         1184 non-null   int64 
 3   employment_type    1184 non-null   object
 4   company_location   1184 non-null   object
 5   is_remote          1184 non-null   bool  
 6   employee_location  1184 non-null   object
 7   job_skills         1184 non-null   object
 8   degree_required    1184 non-null   bool  
 9   posting_date       1184 non-null   object
 10  company_name       1184 non-null   object
 11  salary_tier        1184 non-null   object
 12  has_target         1184 non-null   bool  
 13  degree_flag        1184 non-null   object
dtypes: bool(3), int64(2), object(9)
memory usage: 105.3+ KB
